In [ ]:
from IPython.display import display
import sympy as sp
from sympy import sin, cos

## Lagrange Calculator

### Define Symbols

In [122]:
# Time symbol
t = sp.symbols('t')

# Define functions of time
x = sp.Function('x')(t)
theta = sp.Function('theta')(t)

# First derivatives
dx = sp.diff(x, t)
dtheta = sp.diff(theta, t)

# Second derivatives
ddx = sp.diff(dx, t)
ddtheta = sp.diff(dtheta, t)

In [123]:
# Parameters
r, Mb, Lb, Ib, Mw, Iw, g, tau = sp.symbols('r M_b L_b I_b M_w I_w g tau')

#### Define a Latex Converter For Better Readability

In [142]:
def Latex_Dmath_Converter(expr, header, collect_flag=True):
    # Define symbols and functions
    t, r = sp.symbols('t r')
    x_func = sp.Function('x')(t)
    theta_func = sp.Function('theta')(t)
    phi_func = sp.Function('phi')(t)

    # Define derivatives
    dx = sp.diff(x_func, t)
    ddx = sp.diff(x_func, t, 2)
    dtheta = sp.diff(theta_func, t)
    ddtheta = sp.diff(theta_func, t, 2)
    dphi = sp.diff(phi_func, t)
    ddphi = sp.diff(phi_func, t, 2)

    # LaTeX symbol mapping
    subs_dict = {
        x_func: sp.Symbol('x'),
        theta_func: sp.Symbol(r'\theta'),
        dx: sp.Symbol(r'\dot{x}'),
        ddx: sp.Symbol(r'\ddot{x}'),
        dtheta: sp.Symbol(r'\dot{\theta}'),
        ddtheta: sp.Symbol(r'\ddot{\theta}'),
        phi_func: sp.Symbol(r'\phi'),
        dphi: sp.Symbol(r'\dot{\phi}'),
        ddphi: sp.Symbol(r'\ddot{\phi}')
    }

    # Smart substitution function
    def smart_substitute(expr):
        # Substitution: theta + x/r → phi
        expr = expr.replace(lambda e: e == theta_func + x_func / r, lambda e: phi_func)

        # Derivative substitution
        expr = expr.replace(lambda e: e == dtheta + dx / r, lambda e: dphi)
        expr = expr.replace(lambda e: e == ddtheta + ddx / r, lambda e: ddphi)

        expr = expr.expand()

    # Continue with wild pattern matching as before...


        # Match general a*dx + b*dtheta with a/b == 1/r
        A, B = sp.Wild('A'), sp.Wild('B')
        pattern_dphi = A * dx + B * dtheta
        pattern_ddphi = A * ddx + B * ddtheta

        def replace_dphi(e):
            match = e.match(pattern_dphi)
            if match:
                a_val = match[A]
                b_val = match[B]
                if sp.simplify(a_val / b_val) == 1 / r:
                    return b_val * dphi
            return e

        def replace_ddphi(e):
            match = e.match(pattern_ddphi)
            if match:
                a_val = match[A]
                b_val = match[B]
                if sp.simplify(a_val / b_val) == 1 / r:
                    return b_val * ddphi
            return e

        # Use replace recursively
        expr = expr.replace(pattern_dphi, replace_dphi)
        expr = expr.replace(pattern_ddphi, replace_ddphi)

        # Try substitution within sin/cos or other functions
        expr = expr.replace(lambda e: isinstance(e, sp.sin) and e.args[0] == theta_func + x_func / r,
                            lambda e: sp.sin(phi_func))
        expr = expr.replace(lambda e: isinstance(e, sp.cos) and e.args[0] == theta_func + x_func / r,
                            lambda e: sp.cos(phi_func))

        return expr

    # Process the expression
    if isinstance(expr, sp.Equality):
        lhs = smart_substitute(expr.lhs).subs(subs_dict)
        rhs = smart_substitute(expr.rhs).subs(subs_dict)
        if collect_flag:
            collect_vars = list(subs_dict.values())
            lhs = sp.collect(sp.expand(lhs), collect_vars)
            rhs = sp.collect(sp.expand(rhs), collect_vars)
        latex_str = sp.latex(lhs) + "=" + sp.latex(rhs)
    else:
        expr_subs = smart_substitute(expr).subs(subs_dict)
        if collect_flag:
            collect_vars = list(subs_dict.values())
            expr_subs = sp.collect(sp.expand(expr_subs), collect_vars)
        latex_str = sp.latex(expr_subs)

    # Clean LaTeX output
    latex_str = latex_str.replace("1.0 ", "").replace("1.0", "")

    # Output
    print(r"\begin{multline*}")
    print(header, latex_str)
    print(r"\end{multline*}")


### Kinematic and Potential Terms

In [143]:
T1 = (1/2)*Mw*(dx**2) # Wheel Translation K
T2 = (1/2)*Iw*((dx/r)**2) # Wheel Rotatory K
T3 = (1/2)*Mb*((dx**2)+(2*dx*(dx/r+dtheta)*Lb*cos(x/r+theta))+((dx/r+dtheta)**2)*(Lb**2)) # Body Translation K
T4 = (1/2)*Ib*((dx/r+dtheta)**2)# Wheel Rotatory K
T = T1+T2+T3+T4

In [144]:
V = Mb*g*Lb*cos(x/r+theta)

In [145]:
L = T-V

In [146]:
Latex_Dmath_Converter(T,r"T=",False)
Latex_Dmath_Converter(V,r"T=",False)
Latex_Dmath_Converter(L,r"L=",False)

\begin{multline*}
T= 0.5 I_{b} \dot{\phi}^{2} + \frac{0.5 I_{w} \dot{x}^{2}}{r^{2}} + 0.5 L_{b}^{2} M_{b} \dot{\phi}^{2} + L_{b} M_{b} \dot{\phi} \dot{x} \cos{\left(\phi \right)} + 0.5 M_{b} \dot{x}^{2} + 0.5 M_{w} \dot{x}^{2}
\end{multline*}
\begin{multline*}
T= L_{b} M_{b} g \cos{\left(\phi \right)}
\end{multline*}
\begin{multline*}
L= 0.5 I_{b} \dot{\phi}^{2} + \frac{0.5 I_{w} \dot{x}^{2}}{r^{2}} + 0.5 L_{b}^{2} M_{b} \dot{\phi}^{2} + L_{b} M_{b} \dot{\phi} \dot{x} \cos{\left(\phi \right)} - L_{b} M_{b} g \cos{\left(\phi \right)} + 0.5 M_{b} \dot{x}^{2} + 0.5 M_{w} \dot{x}^{2}
\end{multline*}


### Generalized Coordinates Functions

In [147]:
def coordinate_x(L, RHS):
    dL_ddx = sp.diff(L, dx)
    d_dt_dL_ddx = sp.diff(dL_ddx, t)
    dL_dx = sp.diff(L, x)
    
    lagrange_eq = sp.Eq(d_dt_dL_ddx - dL_dx, RHS)
    return dL_ddx, d_dt_dL_ddx, dL_dx, lagrange_eq


def coordinate_theta(L, RHS):
    dL_ddtheta = sp.diff(L, dtheta)
    d_dt_dL_ddtheta = sp.diff(dL_ddtheta, t)
    dL_dtheta = sp.diff(L, theta)

    lagrange_eq = sp.Eq(d_dt_dL_ddtheta - dL_dtheta, RHS)
    return dL_ddtheta, d_dt_dL_ddtheta, dL_dtheta, lagrange_eq

###  Calculate the diff

In [148]:
dL_ddx, d_dt_dL_ddx, dL_dx, lagrange_eq_x = coordinate_x(L, 0)
dL_ddtheta, d_dt_dL_ddtheta, dL_dtheta, lagrange_eq_theta = coordinate_theta(L, tau)

In [149]:
print(dL_dtheta)

L_b*M_b*g*sin(theta(t) + x(t)/r) - 1.0*L_b*M_b*(Derivative(theta(t), t) + Derivative(x(t), t)/r)*sin(theta(t) + x(t)/r)*Derivative(x(t), t)


#### Print For Latex

In [160]:
Latex_Dmath_Converter(dL_ddx,r"\frac{\partial L}{\partial \dot{x}}=")
Latex_Dmath_Converter(d_dt_dL_ddx, r"\frac{d}{dt}(\frac{\partial L}{\partial \dot{x}})=")
Latex_Dmath_Converter(dL_dx, r"\frac{\partial L}{\partial x}=")
Latex_Dmath_Converter(lagrange_eq_x, r"\text{Coordinate X: }")
Latex_Dmath_Converter(dL_ddtheta, r"\frac{\partial L}{\partial \dot{\theta}}=")
Latex_Dmath_Converter(d_dt_dL_ddtheta, r"\frac{d}{dt}(\frac{\partial L}{\partial \dot{\theta}})=")
Latex_Dmath_Converter(dL_dtheta, r"\frac{\partial L}{\partial \theta}=")
Latex_Dmath_Converter(lagrange_eq_theta, r"\text{Coordinate \theta: }")


\begin{multline*}
\frac{\partial L}{\partial \dot{x}}= \dot{\phi} \left(\frac{I_{b}}{r} + \frac{L_{b}^{2} M_{b}}{r} + L_{b} M_{b} \cos{\left(\phi \right)}\right) + \dot{x} \left(\frac{I_{w}}{r^{2}} + \frac{L_{b} M_{b} \cos{\left(\phi \right)}}{r} + M_{b} + M_{w}\right)
\end{multline*}
\begin{multline*}
\frac{d}{dt}(\frac{\partial L}{\partial \dot{x}})= - L_{b} M_{b} \dot{\phi}^{2} \sin{\left(\phi \right)} - \frac{L_{b} M_{b} \dot{\phi} \dot{x} \sin{\left(\phi \right)}}{r} + \ddot{\phi} \left(\frac{I_{b}}{r} + \frac{L_{b}^{2} M_{b}}{r} + L_{b} M_{b} \cos{\left(\phi \right)}\right) + \ddot{x} \left(\frac{I_{w}}{r^{2}} + \frac{L_{b} M_{b} \cos{\left(\phi \right)}}{r} + M_{b} + M_{w}\right)
\end{multline*}
\begin{multline*}
\frac{\partial L}{\partial x}= - \frac{L_{b} M_{b} \dot{\phi} \dot{x} \sin{\left(\phi \right)}}{r} + \frac{L_{b} M_{b} g \sin{\left(\phi \right)}}{r}
\end{multline*}
\begin{multline*}
\text{Coordinate X: } - L_{b} M_{b} \dot{\phi}^{2} \sin{\left(\phi \right)} - \frac{L_

In [172]:
display(lagrange_eq_x)
display(lagrange_eq_theta)

Eq(1.0*I_b*(Derivative(theta(t), (t, 2)) + Derivative(x(t), (t, 2))/r)/r + 1.0*I_w*Derivative(x(t), (t, 2))/r**2 - L_b*M_b*g*sin(theta(t) + x(t)/r)/r + 1.0*L_b*M_b*(Derivative(theta(t), t) + Derivative(x(t), t)/r)*sin(theta(t) + x(t)/r)*Derivative(x(t), t)/r + 0.5*M_b*(2*L_b**2*(Derivative(theta(t), (t, 2)) + Derivative(x(t), (t, 2))/r)/r - 2*L_b*(Derivative(theta(t), t) + Derivative(x(t), t)/r)**2*sin(theta(t) + x(t)/r) + 2*L_b*(Derivative(theta(t), (t, 2)) + Derivative(x(t), (t, 2))/r)*cos(theta(t) + x(t)/r) - 2*L_b*(Derivative(theta(t), t) + Derivative(x(t), t)/r)*sin(theta(t) + x(t)/r)*Derivative(x(t), t)/r + 2*L_b*cos(theta(t) + x(t)/r)*Derivative(x(t), (t, 2))/r + 2*Derivative(x(t), (t, 2))) + 1.0*M_w*Derivative(x(t), (t, 2)), 0)

Eq(0.5*I_b*(2*Derivative(theta(t), (t, 2)) + 2*Derivative(x(t), (t, 2))/r) - L_b*M_b*g*sin(theta(t) + x(t)/r) + 1.0*L_b*M_b*(Derivative(theta(t), t) + Derivative(x(t), t)/r)*sin(theta(t) + x(t)/r)*Derivative(x(t), t) + 0.5*M_b*(L_b**2*(2*Derivative(theta(t), (t, 2)) + 2*Derivative(x(t), (t, 2))/r) - 2*L_b*(Derivative(theta(t), t) + Derivative(x(t), t)/r)*sin(theta(t) + x(t)/r)*Derivative(x(t), t) + 2*L_b*cos(theta(t) + x(t)/r)*Derivative(x(t), (t, 2))), tau)

### Solve the Equation

In [168]:
solutions = sp.solve([lagrange_eq_x, lagrange_eq_theta], (ddx, ddtheta))
xdd_solution = solutions[ddx]
thetadd_solution = solutions[ddtheta]

In [169]:
Latex_Dmath_Converter(xdd_solution,r"\ddot{x}=")
Latex_Dmath_Converter(thetadd_solution,r"\ddot{\theta}=")

\begin{multline*}
\ddot{x}= - \frac{I_{b} r \tau}{I_{b} I_{w} + I_{b} M_{b} r^{2} + I_{b} M_{w} r^{2} + I_{w} L_{b}^{2} M_{b} - L_{b}^{2} M_{b}^{2} r^{2} \cos^{2}{\left(\phi \right)} + L_{b}^{2} M_{b}^{2} r^{2} + L_{b}^{2} M_{b} M_{w} r^{2}} - \frac{L_{b}^{2} M_{b}^{2} g r^{2} \sin{\left(\phi \right)} \cos{\left(\phi \right)}}{I_{b} I_{w} + I_{b} M_{b} r^{2} + I_{b} M_{w} r^{2} + I_{w} L_{b}^{2} M_{b} - L_{b}^{2} M_{b}^{2} r^{2} \cos^{2}{\left(\phi \right)} + L_{b}^{2} M_{b}^{2} r^{2} + L_{b}^{2} M_{b} M_{w} r^{2}} - \frac{L_{b}^{2} M_{b} r \tau}{I_{b} I_{w} + I_{b} M_{b} r^{2} + I_{b} M_{w} r^{2} + I_{w} L_{b}^{2} M_{b} - L_{b}^{2} M_{b}^{2} r^{2} \cos^{2}{\left(\phi \right)} + L_{b}^{2} M_{b}^{2} r^{2} + L_{b}^{2} M_{b} M_{w} r^{2}} - \frac{L_{b} M_{b} r^{2} \tau \cos{\left(\phi \right)}}{I_{b} I_{w} + I_{b} M_{b} r^{2} + I_{b} M_{w} r^{2} + I_{w} L_{b}^{2} M_{b} - L_{b}^{2} M_{b}^{2} r^{2} \cos^{2}{\left(\phi \right)} + L_{b}^{2} M_{b}^{2} r^{2} + L_{b}^{2} M_{b} M_{w} r^{2}} + \dot

### Linearization

In [173]:
#Define constant tau0
tau0 = sp.symbols('tau0')

# State vector
X = sp.Matrix([x, dx, theta, dtheta])

# Dynamics vector
dX = sp.Matrix([dx, xdd_solution, dtheta, thetadd_solution])

# Equilibrium point
eq_point = {x: 0, dx: 0, theta: 0, dtheta: 0, tau: tau0}

# Compute Jacobians and evaluate at equilibrium
A = dX.jacobian(X).subs(eq_point)
B = dX.jacobian([tau]).subs(eq_point)

# Print A matrix in LaTeX
print("A matrix in LaTeX:")
print(sp.latex(A))

# Print B matrix in LaTeX
print("\nB matrix in LaTeX:")
print(sp.latex(B))

A matrix in LaTeX:
\left[\begin{matrix}0 & 1 & 0 & 0\\- \frac{L_{b}^{2} M_{b}^{2} g r}{I_{b} I_{w} + I_{b} M_{b} r^{2} + I_{b} M_{w} r^{2} + I_{w} L_{b}^{2} M_{b} + L_{b}^{2} M_{b} M_{w} r^{2}} & 0 & - \frac{L_{b}^{2} M_{b}^{2} g r^{2}}{I_{b} I_{w} + I_{b} M_{b} r^{2} + I_{b} M_{w} r^{2} + I_{w} L_{b}^{2} M_{b} + L_{b}^{2} M_{b} M_{w} r^{2}} & 0\\0 & 0 & 0 & 1\\\frac{I_{w} L_{b} M_{b} g}{I_{b} I_{w} r + I_{b} M_{b} r^{3} + I_{b} M_{w} r^{3} + I_{w} L_{b}^{2} M_{b} r + L_{b}^{2} M_{b} M_{w} r^{3}} + \frac{L_{b}^{2} M_{b}^{2} g r}{I_{b} I_{w} r + I_{b} M_{b} r^{3} + I_{b} M_{w} r^{3} + I_{w} L_{b}^{2} M_{b} r + L_{b}^{2} M_{b} M_{w} r^{3}} + \frac{L_{b} M_{b}^{2} g r^{2}}{I_{b} I_{w} r + I_{b} M_{b} r^{3} + I_{b} M_{w} r^{3} + I_{w} L_{b}^{2} M_{b} r + L_{b}^{2} M_{b} M_{w} r^{3}} + \frac{L_{b} M_{b} M_{w} g r^{2}}{I_{b} I_{w} r + I_{b} M_{b} r^{3} + I_{b} M_{w} r^{3} + I_{w} L_{b}^{2} M_{b} r + L_{b}^{2} M_{b} M_{w} r^{3}} & 0 & \frac{I_{w} L_{b} M_{b} g r}{I_{b} I_{w} r + I_{b} M_{b} r